Probing YandexGPTLite model. Each question to be probed 10 times in a row in isolation from any context.

Docs:
https://yandex.cloud/ru/docs/foundation-models/operations/yandexgpt/create-prompt

https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/

In [5]:
import json

with open('api_keys.json') as json_file:
    apikeys = json.load(json_file)

In [2]:
import requests

oauth = apikeys['yandexgpt_oauth']
folder = apikeys['yandexgpt_folder']

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
data = { 'yandexPassportOauthToken': oauth }

response = requests.post(url, json=data)

auth = response.json()['iamToken']

In [ ]:
import pandas as pd

df_prompts = pd.read_csv('prompts.csv', sep=';', encoding='utf-8-sig')

df_prompts

Subsequently calling the api might lead to throttle.

In [3]:
def get_response_from_yandexgpt(prompt):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {auth}",
        "x-folder-id": folder
    }
    message = {
      "modelUri": f"gpt://{folder}/yandexgpt-lite/rc",
      "completionOptions": {
        "stream": False,
        "temperature": 0.3
      },
      "messages": [
        {
          "role": "user",
          "text": prompt
        }
      ]
    }

    response = requests.post(url, headers=headers, json=message)
    return response.json()['result']['alternatives'][0]['message']['text']

In [4]:
import time
responses = []

for prompt in df_prompts['prompt']:
    for i in range(10):
        message = get_response_from_yandexgpt(prompt)
        responses.append((prompt, message))
        time.sleep(0.5)

In [ ]:
df_responses = pd.DataFrame(responses, columns=['question','response'])

df_responses.to_csv('responses_yandexGPT.csv', sep=';', encoding='utf-8-sig')

df_responses